# 🚀 Stage 1 Training on Google Colab A100

This notebook trains the Stage 1 optimized model (ConvNeXt-Base + 512px) on Google Colab with A100 GPU.

**Expected performance**: 85-87% Macro-F1 (up from 80.1% baseline)

**Training time**: ~2 hours on A100 (vs 4-5 hours on RTX 3050)

## Step 1: Setup Environment

⚠️ **Important**: Make sure you have selected **A100 GPU** in Runtime > Change runtime type

In [ ]:
# Check GPU type
!nvidia-smi --query-gpu=name,memory.total --format=csv

## Step 2: Clone Repository

In [ ]:
# Clone your GitHub repository
!git clone https://github.com/thc1006/nycu-CSIC30014-LAB3.git
%cd nycu-CSIC30014-LAB3

## Step 3: Install Dependencies

In [ ]:
%%bash
pip install -q --upgrade pip
# PyTorch with CUDA 12.1 (compatible with Colab as of Oct 2025)
pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
pip install -q -r requirements.txt

## Step 4: Mount Google Drive and Setup Data

**Before running this cell**, make sure you have uploaded your image directories to Google Drive:

```
MyDrive/chest-xray-data/
  ├── train_images/
  ├── val_images/
  └── test_images/
```

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Step 5: Update Config for Colab Paths

In [ ]:
import yaml

# Update config paths for Colab
config_path = 'configs/model_stage1.yaml'

# Read base config
with open('configs/base.yaml', 'r') as f:
    base_config = yaml.safe_load(f)

# Read stage1 config
with open(config_path, 'r') as f:
    stage1_config = yaml.safe_load(f)

# Update paths for Colab + Google Drive
base_config['data']['images_dir_train'] = '/content/drive/MyDrive/chest-xray-data/train_images'
base_config['data']['images_dir_val'] = '/content/drive/MyDrive/chest-xray-data/val_images'
base_config['data']['images_dir_test'] = '/content/drive/MyDrive/chest-xray-data/test_images'
base_config['data']['train_csv'] = 'data/train_data.csv'
base_config['data']['val_csv'] = 'data/val_data.csv'
base_config['data']['test_csv'] = 'data/test_data.csv'
base_config['out']['submission_path'] = 'submission_stage1_colab.csv'

# Increase batch size for A100
if 'train' not in stage1_config:
    stage1_config['train'] = {}
stage1_config['train']['batch_size'] = 24  # A100 can handle larger batches

# Save updated base config
with open('configs/base.yaml', 'w') as f:
    yaml.dump(base_config, f)

# Save updated stage1 config
with open(config_path, 'w') as f:
    yaml.dump(stage1_config, f)

print("✓ Config updated for Colab")
print(f"  Train images: {base_config['data']['images_dir_train']}")
print(f"  Batch size: {stage1_config['train']['batch_size']}")

## Step 6: Verify CUDA and Enable Optimizations

In [ ]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Enable TF32 for A100 (significant speedup)
torch.set_float32_matmul_precision('high')
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True

print("\n✓ Optimizations enabled:")
print(f"  TF32 matmul: {torch.backends.cuda.matmul.allow_tf32}")
print(f"  TF32 cuDNN: {torch.backends.cudnn.allow_tf32}")
print(f"  cuDNN benchmark: {torch.backends.cudnn.benchmark}")

## Step 7: Generate test_data.csv (if needed)

In [ ]:
import os

if not os.path.exists('data/test_data.csv'):
    print("Generating test_data.csv...")
    !python -m src.build_test_csv --config configs/model_stage1.yaml
else:
    print("✓ test_data.csv already exists")

## Step 8: Run Component Tests

In [ ]:
# Quick test to verify everything works
!python test_stage1.py

## Step 9: Start Training! 🚀

This will take approximately **2 hours** on A100.

You'll see output like:
```
[epoch 01] train acc=0.3500 f1=0.2800 | val acc=0.4500 f1=0.3500
[epoch 10] train acc=0.8000 f1=0.7800 | val acc=0.7900 f1=0.7700
[epoch 20] train acc=0.8800 f1=0.8700 | val acc=0.8500 f1=0.8400
[epoch 30] train acc=0.9200 f1=0.9100 | val acc=0.8700 f1=0.8600
[SWA final] val acc=0.8750 f1=0.8650
```

In [ ]:
!python -m src.train_v2 --config configs/model_stage1.yaml

## Step 10: Evaluate Model

In [ ]:
# Evaluate and generate confusion matrix
!python -m src.eval --config configs/model_stage1.yaml --ckpt outputs/stage1_convnext512/best_swa.pt

## Step 11: Generate Predictions with TTA

Test-Time Augmentation will give us an additional **+2-3% boost**

In [ ]:
# Generate predictions with Test-Time Augmentation
!python -m src.tta_predict --config configs/model_stage1.yaml --ckpt outputs/stage1_convnext512/best_swa.pt

## Step 12: Download Results

In [ ]:
from google.colab import files

# Download submission file
files.download('submission_stage1_colab.csv')

# Optional: Download model checkpoint
# files.download('outputs/stage1_convnext512/best_swa.pt')

## 🎉 Training Complete!

### Expected Results:

- **Validation Macro-F1**: 0.86-0.87
- **Expected Public Score**: 0.87-0.89 (with TTA)
- **Improvement**: +7-9% from baseline (0.801)

### Per-Class F1 Improvements:

| Class | Baseline | Expected | Improvement |
|-------|----------|----------|-------------|
| Normal | 0.897 | 0.930-0.950 | +3-5% |
| Bacteria | 0.762 | 0.840-0.870 | +8-11% |
| **Virus** | 0.619 | 0.800-0.840 | **+18-22%** |
| COVID-19 | 0.875 | 0.920-0.970 | +5-10% |

### Next Steps:

1. **Submit to Kaggle** and check your Public Score
2. **Stage 2**: Train ensemble of 3 models for 88-90%
3. **Stage 3**: Multi-scale training for 90-93%

---

**Questions or issues?** Check the documentation:
- `START_HERE.md` - Quick start guide
- `RUN_STAGE1.md` - Detailed technical docs
- `data/README.md` - Data setup instructions